Contract details
-------------------

In [1]:
from ib_insync import *
util.startLoop()

import logging
# util.logToConsole(logging.DEBUG)

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=11,readonly=True)

<IB connected to 127.0.0.1:7497 clientId=11>

Suppose we want to find the contract details for AMD stock.
Let's create a stock object and request the details for it:

In [3]:
amd = Stock('AMD')

cds = ib.reqContractDetails(amd)

len(cds)

36

We get a long list of contract details. Lets print the first one:

In [5]:
cds

E,PSX', priceMagnifier=1, underConId=0, longName='ADVANCED MICRO DEVICES', contractMonth='', industry='Technology', category='Semiconductors', subcategory='Electronic Compo-Semicon', timeZoneId='EST (Östliche Normalzeit)', tradingHours='20200910:0800-20200910:1700;20200911:0800-20200911:1700;20200912:CLOSED;20200913:CLOSED;20200914:0800-20200914:1700;20200915:0800-20200915:1700', liquidHours='20200910:0800-20200910:1700;20200911:0800-20200911:1700;20200912:CLOSED;20200913:CLOSED;20200914:0800-20200914:1700;20200915:0800-20200915:1700', evRule='', evMultiplier=0, mdSizeMultiplier=100, aggGroup=1, underSymbol='', underSecType='', marketRuleIds='26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26', secIdList=[], realExpirationDate='', lastTradeTime='', stockType='COMMON', cusip='', ratings='', descAppend='', bondType='', couponType='', callable=False, putable=False, coupon=0, convertible=False, maturity='', issueDate='', nextOptionDate='', nextOptionType='', nextOptionPartia

The contract itself is in the 'contract' property of the contract details. Lets make a list of contracts and look at the first:

In [6]:
contracts = [cd.contract for cd in cds]

contracts[0]

Contract(secType='STK', conId=4391, symbol='AMD', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMD', tradingClass='NMS')

To better spot the difference between all the contracts it's handy to convert to a DataFrame. There is a utility function to do that:

In [7]:
util.df(contracts)

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,STK,4391,AMD,,0.0,,,SMART,NASDAQ,USD,AMD,NMS,False,,,,[],None
1,STK,32596680,AMD,,0.0,,,SMART,IBIS,EUR,AMD,XETRA,False,,,,[],None
2,STK,172603743,AMD,,0.0,,,SMART,EBS,CHF,AMD,AMD,False,,,,[],None
3,STK,298633607,AMD,,0.0,,,SMART,ASX,AUD,AMD,AMD,False,,,,[],None
4,STK,4391,AMD,,0.0,,,AMEX,NASDAQ,USD,AMD,NMS,False,,,,[],None
5,STK,4391,AMD,,0.0,,,NYSE,NASDAQ,USD,AMD,NMS,False,,,,[],None
6,STK,4391,AMD,,0.0,,,CBOE,NASDAQ,USD,AMD,NMS,False,,,,[],None
7,STK,4391,AMD,,0.0,,,PHLX,NASDAQ,USD,AMD,NMS,False,,,,[],None
8,STK,4391,AMD,,0.0,,,ISE,NASDAQ,USD,AMD,NMS,False,,,,[],None
9,STK,4391,AMD,,0.0,,,CHX,NASDAQ,USD,AMD,NMS,False,,,,[],None


We can see from this that AMD trades in different currencies on different exchanges.
Suppose we want the one in USD on the SMART exchange. The AMD contract is adjusted to
reflect that and becomes unique:

In [8]:
amd = Stock('AMD', 'SMART', 'USD')

assert len(ib.reqContractDetails(amd)) == 1

Lets try the same for Intel:

In [9]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

Let's try a non-existing contract:

In [10]:
xxx = Stock('XXX', 'SMART', 'USD')

assert len(ib.reqContractDetails(xxx)) == 0

Error 200, reqId 7: No security definition has been found for the request, contract: Stock(symbol='XXX', exchange='SMART', currency='USD')


or a Forex contract

In [11]:
eurusd = Forex('EURUSD')

assert len(ib.reqContractDetails(eurusd)) == 1

With the ``qualifyContracts`` method the extra information that is send back
from the contract details request is used to fill in the original contracts.

Lets do that with ``amd`` and compare before and aftwards:

In [12]:
amd

Stock(symbol='AMD', exchange='SMART', currency='USD')

In [15]:
ib.qualifyContracts(amd)
amd

Stock(conId=4391, symbol='AMD', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMD', tradingClass='NMS')

**TIP:** When printing a contract, the output can be copy-pasted and it will be valid Python code.

The ``conId`` that is returned can by itself be used to uniquely specify a contract:

In [16]:
contract_4391 = Contract(conId=4391)

ib.qualifyContracts(contract_4391)

assert contract_4391 == amd

A whole bunch of contracts can be qualified at the same time. A list of all the successfull ones is returned:

In [17]:
qualContracts = ib.qualifyContracts(amd, intc, xxx, eurusd)

assert intc in qualContracts
assert xxx not in qualContracts

Error 200, reqId 15: No security definition has been found for the request, contract: Stock(symbol='XXX', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='XXX', exchange='SMART', currency='USD')


There is also an API function to request stocks (only stocks) that match a pattern:

In [18]:
matches = ib.reqMatchingSymbols('intc')
matchContracts = [m.contract for m in matches]

matches

[ContractDescription(contract=Contract(secType='STK', conId=270639, symbol='INTC', primaryExchange='NASDAQ.NMS', currency='USD'), derivativeSecTypes=['CFD', 'OPT', 'IOPT', 'WAR', 'FUT', 'BAG']),
 ContractDescription(contract=Contract(secType='STK', conId=12178234, symbol='INTC', primaryExchange='EBS', currency='CHF'), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=38709539, symbol='INTC', primaryExchange='MEXI', currency='MXN'), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=11463493, symbol='INTCUSD', primaryExchange='EBS', currency='USD'), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=130861844, symbol='S', primaryExchange='MEXI', currency='MXN'), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=130861851, symbol='2S7', primaryExchange='VALUE', currency='EUR'), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', c

In [19]:
assert intc in matchContracts

In [20]:
ib.disconnect()